In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import torch.nn as nn
import json
import os
import random
import torch

from collections import Counter, defaultdict
from itertools import islice
from sklearn.utils import shuffle
from tqdm import tqdm, trange
from torch.optim import Adam
from torch.utils.data import TensorDataset

from models.Encoder import *
from models.Decoder import *
from models.utils import *
from utils import *

# set fixed random seeds to reproduce results
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

%matplotlib inline

# Experiment 3

In [ ]:
# define experiment
exp='/exp_3'

# define subexperiment
subexp= '/jump' #'/turn_left'

# define number of iterations
n_iters = 20000

# define batch size
batch_size = 32

In [ ]:
device = ("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# experiment 3 #

# load dataset into memory, and get w2idx, idx2w, w2freq dictionaries and lists of input and output sentences
cmd_vocab, w2i_cmds, i2w_cmds, cmds_train, act_vocab, w2i_acts, i2w_acts, acts_train = load_dataset(exp=exp, 
                                                                                                    split='/train', 
                                                                                                    subexp=subexp)
_, _, _, cmds_test, _, _, _, acts_test = load_dataset(exp=exp, 
                                                      split='/test', 
                                                      subexp=subexp)

In [ ]:
## create input and output language pairs ##

# training
train_cmd_act_pairs = create_pairs(cmds_train, acts_train)
print("Number of train source-target pairs: {}".format(len(train_cmd_act_pairs)))

# testing
test_cmd_act_pairs = create_pairs(cmds_test, acts_test)
print("Number of test source-target pairs: {}".format(len(test_cmd_act_pairs)))

In [ ]:
# show random train command-action pair
random_pair = random.choice(train_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

In [ ]:
# show random test command-action pair
random_pair = random.choice(test_cmd_act_pairs)
print("Command: {}".format(random_pair[0]))
print("Action: {}".format(random_pair[1]))

## Prepare data

In [ ]:
cmds_train, acts_train, input_lengths_train, masks = pairs2idx(cmds_train, acts_train, w2i_cmds, w2i_acts, padding=True, training=True)
cmds_test, acts_test, input_lengths_test = pairs2idx(cmds_test, acts_test, w2i_cmds, w2i_acts, padding=True, training=False)

In [ ]:
# create train and test data loaders
train_dl = create_batches(cmds_train, acts_train, input_lengths_train, batch_size=batch_size, masks=masks, split='train', num_samples=n_iters)
test_dl = create_batches(cmds_test, acts_test, input_lengths_test, batch_size=batch_size, split='test')

In [ ]:
### Hyperparameters for training ###

# source language (i.e., commands) vocabulary size |V_source|
in_size = len(w2i_cmds)

# target language (i.e., actions) vocabulary size |V_target|
out_size = len(w2i_acts)

# size of word embeddings
emb_size = 20 #[10, 20]

# size of hidden units
hidden_size = 50 #[50, 100]

# number of layers
layer_size = 2 #[1, 2]

# layer dropout rates
dropout_rate = 0.5 #[0.25, 0.5]

# learning rate
lr = 1e-3

# define whether encoder is uni- or bidirectional (decoder cannot be bidirectional)
bidir = True

# number of epochs
n_epochs = 12 # 10-15 epochs (20.000 iterations each) seem to be sufficient to let models converge (find local minima)

## Grid search over specified hyperparameter space

In [ ]:
def grid_search(in_size:int, out_size:int, embedding_sizes:list, hidden_sizes:list, layer_sizes:list, 
                dropout_rates:list, lr:float, n_epochs:int):
    batch_size = 32
    grid_results = defaultdict(dict)
    for emb_size in embedding_sizes:
        for hidden_size in hidden_sizes:
            for layer_size in layer_sizes:
                for dropout_rate in dropout_rates:
                    # instantiate models
                    encoder = EncoderGRU(in_size, emb_size, hidden_size, layer_size, dropout=dropout_rate, bidir=False)
                    decoder = DecoderGRU(emb_size, hidden_size, out_size, layer_size, dropout=dropout_rate)
                    # move models to GPU, if nvidia GPU is available (faster computation)
                    encoder.cuda()
                    decoder.cuda()
                    # train
                    train_losses, train_accs, encoder, decoder = train(train_dl, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts,
                                                                       encoder, decoder, epochs=n_epochs,
                                                                       batch_size=batch_size, learning_rate=lr,
                                                                       detailed_analysis=False)
                    # test
                    test_acc = test(test_dl, w2i_cmds, w2i_acts, i2w_cmds, i2w_acts, encoder, decoder,
                                    batch_size=batch_size, detailed_analysis=True, detailed_results=False)
                    # store results (convert keys to str to save results as .json file)
                    grid_results[str((emb_size, hidden_size, layer_size, dropout_rate))]['train_accs'] = train_accs
                    grid_results[str((emb_size, hidden_size, layer_size, dropout_rate))]['train_losses'] = train_losses
                    grid_results[str((emb_size, hidden_size, layer_size, dropout_rate))]['test_acc'] = test_acc
    return grid_results

In [ ]:
grid_results = grid_search(in_size, out_size, embedding_sizes, hidden_sizes, layer_sizes, dropout_rates, lr, n_epochs)

In [ ]:
# save results in .json file
with open('./results/experiment_2_GRU_grid_search.json', 'w') as json_file:
      json.dump(grid_results_copy, json_file)

In [ ]:
test_accs = {hypers: results['test_acc'] for hypers, results in grid_results.items()}
test_accs = dict(sorted(test_accs.items(), key=lambda kv:kv[1], reverse=False))

## Debugging

In [ ]:
## Instantiate models ##

encoder = EncoderGRU(in_size, emb_size, hidden_size, layer_size, dropout=dropout_rate, bidir=bidir)
decoder = AttnDecoderGRU(emb_size, hidden_size, out_size, layer_size, dropout=dropout_rate, attention_version='multiplicative')

In [ ]:
encoder.cuda()

In [ ]:
# move models to GPU, if GPU is available (for faster computation)
decoder.cuda()

### Training

In [ ]:
train_losses, train_accs, encoder, decoder = train(train_dl,
                                                   w2i_cmds, w2i_acts,
                                                   i2w_cmds, i2w_acts,
                                                   encoder, decoder,
                                                   epochs= n_epochs,
                                                   batch_size=batch_size,
                                                   learning_rate=lr,
                                                   detailed_analysis=False,
                                                   detailed_results=False,
                                                   similarity_computation=False)

### Testing

In [ ]:
test_acc = test(test_dl,
                w2i_cmds, w2i_acts, 
                i2w_cmds, i2w_acts,
                encoder, decoder,
                batch_size=batch_size,
                detailed_analysis=False,
                detailed_results=False,
                components_accuracy=False)

## Display per component accuracy

In [ ]:
print(results_per_component)

In [ ]:
n = 5
top_and_bottom_n = dict(list(results_per_component.items())[:n] + list(results_per_component.items())[-n:])
plt.bar(top_and_bottom_n.keys(), top_and_bottom_n.values(), alpha=0.5, edgecolor='black')
plt.xlabel('Command components', fontsize=12)
plt.ylabel('Test accuracy (%)', fontsize=12)
plt.yticks(np.arange(0, 120, 20))
plt.xticks(ticks=range(len(top_and_bottom_n)), labels=list(top_and_bottom_n.keys()), rotation=80)
plt.show()

### Show cosine similarity between different phrases

In [ ]:
command = 'run'
nearest_neighbours = compute_similarities(command_hiddens, command)
df = pd.DataFrame.from_dict(nearest_neighbours, orient='index', columns=[command])
df.head()